In [7]:
import json
import logging
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
import os
from os.path import join
import sys
import pickle
import csv
from transformers import BertTokenizer,BertForSequenceClassification
from transformers.optimization import AdamW

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(6)

In [9]:
# set dimension of interest
# dim = 'romance'
dims = [
        'social_support',
        'conflict',
        'trust',
        'fun',
        'similarity',
        'identity',
        'respect',
        'romance',
        'knowledge',
        'power'
        ]

In [10]:
# DATA_DIR = 'data/%s'%dim # directory of where the train/test/dev files are stored
# OUTPUT_DIR = 'weights/BERT/%s' %dim # where the model weights will be stored
BERT_MODEL = 'bert-base-cased' # BERT model type
CACHE_DIR = 'cache/' # where BERT will look for pre-trained models to load parameters from

num_labels = 2

OUTPUT_MODE = 'classification'
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [11]:
# load model
model = BertForSequenceClassification.from_pretrained(BERT_MODEL,cache_dir=CACHE_DIR, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import json
## load sentences
file = open('C:\\Users\\fonta\\PycharmProjects\\10_dimensions_classifier\\data\\examples.txt')
examples = json.load(file)
scoring = {}
for id in examples:
    sentence = examples[id]
    scoring.update({sentence: {}})
    print(f'\n{id}: {sentence}')
    for dim in dims:
        DATA_DIR = 'data/%s'%dim
        OUTPUT_DIR = 'weights/BERT/%s' %dim
        # load pretrained BERT model for specific dimension
        output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
        model.load_state_dict(torch.load(output_model_file))
        model.to(device)
        tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)

        input_ids = torch.tensor([tokenizer.encode(sentence,add_special_tokens=True)]).cuda()
        with torch.no_grad():
            outputs = model(input_ids)[0]
            score = torch.softmax(outputs,1)
        scoring[sentence].update({dim: round(score[0,1].item(), 3)})
        print("\t%s:\t%1.3f" % (dim,score[0,1].item()))

with open('C:\\Users\\fonta\\PycharmProjects\\10_dimensions_classifier\\out\\examples_scoring.txt', 'w') as convert_file:
     convert_file.write(json.dumps(scoring))


1: Well... listen, if you need any help, you know, back up, call me, OK?
	social_support:	0.927
	conflict:	0.008
	trust:	0.026
	fun:	0.024
	similarity:	0.038
	identity:	0.003
	respect:	0.009
	romance:	0.000
	knowledge:	0.003
	power:	0.011

2: Forgive me for askin', son, and I don't mean to belabor the obvious, but why is it that you've got your head so far up your own ass?
	social_support:	0.078
	conflict:	0.970
	trust:	0.026
	fun:	0.025
	similarity:	0.031
	identity:	0.003
	respect:	0.168
	romance:	0.000
	knowledge:	0.003
	power:	0.011

3: I'm trying to tell you - and this is where you have to trust me - but, I think your life might be in real danger
	social_support:	0.535
	conflict:	0.046
	trust:	0.309
	fun:	0.024
	similarity:	0.012
	identity:	0.003
	respect:	0.002
	romance:	0.001
	knowledge:	0.006
	power:	0.011

4: It's just funny...who needs a serial psycho in the woods with a chainsaw when we have ourselves
	social_support:	0.002
	conflict:	0.978
	trust:	0.027
	fun:	0.025
	similar